## Setup

Load Python modules.

In [3]:
!pip install pyspellchecker

from spellchecker import SpellChecker
import numpy as np
import pandas as pd
import os
from tensorflow.python.client import device_lib
import tensorflow as tf
from tensorflow.keras.layers import Embedding,Dropout,Lambda,Dense,Input,LSTM,Concatenate,Flatten,Add,Reshape,GlobalAveragePooling1D,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
import sys
assert sys.version_info[0] >= 3
print(tf.VERSION)
print("Set up!")

     |████████████████████████████████| 1.9MB 3.4MB/s 
1.13.1
Set up!


## Preprocessing

For the preprocessing step, we will create two dictionaries. One will be used to map questions to articles, while the other will be used to map questions and the contents of the articles to the answers.

### Loading JSON datasets

In [4]:
!wget https://github.com/moonman239/Capstone-project/raw/master/data.zip -O data.zip
!unzip data.zip
assert os.path.isfile("train-v1.1.json"),"Non-existent file"

--2019-06-11 19:28:03--  https://github.com/moonman239/Capstone-project/raw/master/data.zip
Resolving github.com (github.com)... 192.30.253.112
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/moonman239/Capstone-project/master/data.zip [following]
--2019-06-11 19:28:04--  https://raw.githubusercontent.com/moonman239/Capstone-project/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9048060 (8.6M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]   8.63M  --.-KB/s    in 0.1s    

2019-06-11 19:28:05 (77.9 MB/s) - ‘data.zip’ saved [9048060/9048060]

Archive:  data.zip
  inflating: dev-v1.1.json    

In [0]:
import json
import re
regex = re.compile(r'\W+')
def readFile(filename):
  with open(filename) as file:
    fields = []
    JSON = json.loads(file.read())
    for article in JSON["data"]:
      articleTitle = article["title"]
      for paragraph in article["paragraphs"]:
        paragraphContext = paragraph["context"]
        for qas in paragraph["qas"]:
          question = qas["question"]
          for answer in qas["answers"]:
            fields.append({"question":question,"answer_text":answer["text"],"answer_start":answer["answer_start"],"paragraph_context":paragraphContext,"article_title":articleTitle})
  fields = pd.DataFrame(fields)
  fields["question"] = fields["question"].str.replace(regex," ")
  assert not (fields["question"].str.contains("catalanswhat").any())
  fields["paragraph_context"] = fields["paragraph_context"].str.replace(regex," ")
  fields["answer_text"] = fields["answer_text"].str.replace(regex," ")
  assert not (fields["paragraph_context"].str.contains("catalanswhat").any())
  fields["article_title"] = fields["article_title"].str.replace("_"," ")
  assert not (fields["article_title"].str.contains("catalanswhat").any())
  return fields

In [0]:
trainingData = readFile("train-v1.1.json")


In [0]:
trainingData

,answer_start,answer_text,article_title,paragraph_context,question
0,515,Saint Bernadette Soubirous,University of Notre Dame,Architecturally the school has a Catholic char...,To whom did the Virgin Mary allegedly appear i...
1,188,a copper statue of Christ,University of Notre Dame,Architecturally the school has a Catholic char...,What is in front of the Notre Dame Main Building
2,279,the Main Building,University of Notre Dame,Architecturally the school has a Catholic char...,The Basilica of the Sacred heart at Notre Dame...
3,381,a Marian place of prayer and reflection,University of Notre Dame,Architecturally the school has a Catholic char...,What is the Grotto at Notre Dame
4,92,a golden statue of the Virgin Mary,University of Notre Dame,Architecturally the school has a Catholic char...,What sits on top of the Main Building at Notre...
5,248,September 1876,University of Notre Dame,As at most other universities Notre Dame s stu...,When did the Scholastic Magazine of Notre dame...
6,441,twice,University of Notre Dame,As at most other universities Notre Dame s stu...,How often is Notre Dame s the Juggler published
7,598,The Observer,University of Notre Dame,As at most other universities Notre Dame s stu...,What is the daily student paper at Notre Dame ...
8,126,three,University of Notre Dame,As at most other universities Notre Dame s stu...,How many student news papers are found at Notr...
9,908,1987,University of Notre Dame,As at most other universities Notre Dame s stu...,In what year did the student paper Common Sens...


In [0]:

trainingData["question"] = trainingData["question"].str.lower()
trainingData["article_title"] = trainingData["article_title"].str.lower()
trainingData["paragraph_context"] = trainingData["paragraph_context"].str.lower()
trainingData["answer_text"] = trainingData["answer_text"].str.lower()
trainingData["answer_start"] = pd.to_numeric(trainingData["answer_start"])

In [8]:
devData = readFile("dev-v1.1.json")
devData["question"] = devData["question"].str.lower()
devData["article_title"] = devData["article_title"].str.lower()
devData["paragraph_context"] = devData["paragraph_context"].str.lower()
devData["answer_text"] = devData["answer_text"].str.lower()
devData["answer_start"] = pd.to_numeric(devData["answer_start"])
print("Finished loading dev data and lowering appropriate columns.")

Finished loading dev data and lowering appropriate columns.


#### Spell- and grammar-checking

In [0]:
def checkMisspellings(sentences):
  spell_checker = SpellChecker(distance=1)
  words = sentences.str.split().values
  unknown = np.vectorize(spell_checker.unknown)
  misspelledWords = unknown(words)
  def correctionFn(_list):
    try:
      corrections = []
      for word in _list:
        assert isinstance(word,str),"String expected."
        corrections.append(spell_checker.correction(word))
      return corrections
    except:
      import sys
      print(sys.exc_info())
  correctionFn = np.vectorize(correctionFn,otypes=[list])
  corrections = correctionFn(np.array(["hello","me"]))
  #words_with_corrections = np.hstack((misspelledWords,corrections))
  return 

In [161]:

test = pd.Series(trainingData["answer_text"][0:2])
checkMisspellings(test)
from cProfile import run
testResults = run('checkMisspellings(test)')
print(testResults)

         121467 function calls (121203 primitive calls) in 0.171 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:997(_handle_fromlist)
        1    0.000    0.000    0.169    0.169 <ipython-input-154-085cbcb8bb29>:1(checkMisspellings)
        2    0.000    0.000    0.000    0.000 <ipython-input-154-085cbcb8bb29>:6(correctionFn)
        1    0.002    0.002    0.171    0.171 <string>:1(<module>)
        1    0.000    0.000    0.079    0.079 __init__.py:302(loads)
        1    0.000    0.000    0.000    0.000 __init__.py:517(__init__)
        2    0.000    0.000    0.013    0.006 __init__.py:586(update)
        1    0.000    0.000    0.000    0.000 _bootlocale.py:23(getpreferredencoding)
        2    0.000    0.000    0.000    0.000 _compression.py:12(_check_not_closed)
        1    0.000    0.000    0.000    0.000 _compression.py:150(tell)
        1  

In [0]:
misspelledWords = checkMisspellings(trainingData["answer_text"])

#### Statistics

In [0]:
from sys import getsizeof
def vocabulary():
  from sklearn.feature_extraction.text import CountVectorizer
  data_frame = trainingData + devData
  data_frame = data_frame.astype("str")
  phrases = []
  for idx,row in data_frame.iterrows():
    phrases.append(row["question"])
    phrases.append(row["paragraph_context"])
    phrases.append(row["article_title"])
  for string in phrases:
    assert "catalanswhat" not in str(string)
  words = CountVectorizer().fit(phrases).get_feature_names()
  assert "raisedin" not in words
  return words
def vocabularySize():
  return len(vocabulary())
def summaryStatistics(series):
    numberOfWords = series.apply(lambda x: len(str(x).split(" ")))
    averageNumberOfWords = sum(numberOfWords) / len(numberOfWords)
    return "average: " + str(averageNumberOfWords) + "maximum: " + str(max(numberOfWords)) + " minimum: " +str(min(numberOfWords))
print("Size of vocabulary: ", vocabularySize())
print("Words in each question: ",summaryStatistics(trainingData["question"]))
print("Words in each article title: ",summaryStatistics(trainingData["article_title"]))
print("Words in each context: ",summaryStatistics(trainingData["paragraph_context"]))
print("Words in each answer: ",summaryStatistics(trainingData["answer_text"]))

AssertionError: ignored

####Get maximum ** possible** answer start.



In [0]:

max_possible_answer_start = int(pd.concat((trainingData["answer_text"],devData["answer_text"])).str.len().max())


####Metrics

In [0]:
from tensorflow.keras import backend as K
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### Integer encode text

Used for manual encoding of text into integers.

In [0]:
strings = pd.concat((trainingData,devData)).drop("answer_start",axis=1)
strings = strings.values.flatten()
textTokenizer = Tokenizer()
textTokenizer.fit_on_texts(strings)

In [0]:
# Get length of vocabulary.
vocabulary_length = len(textTokenizer.word_index) + 1

In [0]:
# Get maximum length of all questions and contexts.
max_length_questions = pd.concat((trainingData["question"],devData["question"])).str.split().len().max
max_length_context = pd.concat((trainingData["paragraph_context"],devData["paragraph_context"])).str.len().max()

AttributeError: ignored

In [0]:
questionsTokenized_train = pad_sequences(np.array(textTokenizer.texts_to_sequences(trainingData["question"])),maxlen=max_length_questions)
contextTokenized_train = pad_sequences(np.array(textTokenizer.texts_to_sequences(trainingData["paragraph_context"])),maxlen=max_length_context)

In [0]:
questionsTokenized_dev = textTokenizer.texts_to_sequences(devData["question"])
contextTokenized_dev = textTokenizer.texts_to_sequences(devData["paragraph_context"])

In [0]:
# Pad sequences.
questionsTokenized_train = pad_sequences(questionsTokenized_train,maxlen=max_length_questions)
contextTokenized_train = pad_sequences(contextTokenized_train,maxlen=max_length_context)

## Second neural network - non-recurrent/no transfer learning

This neural network is used to generate answers from the questions and articles. It works by first reading the relevant article and using the question to find the answer.

In [0]:
#Build the neural network.
from math import log
inputShape_second = X_2_train_num.shape[1:3]
print(inputShape_second)
answers_shape = Y_2_train_num.shape
print(answers_shape[1])
# Find the vocabulary length.
vocabularyLength = np.concatenate((X_2_train_num,X_2_dev_num)).max() + 1

NameError: ignored

In [0]:
questions = Input(shape=(questionsTrain.shape[1],))
context = Input(shape=(contextTrain.shape[1],))
embedding_1 = Embedding(vocabularyLength,16)(questions)
#answers_network.add(Dense(16))
flatten_1 = Flatten()(embedding_1)
hidden = Dense(16)(Dense(8)(flatten_1))
hidden_2 = Dense(16)(hidden)
hidden_3 = Dense(16)(hidden_2)
dropout = Dropout(0.45)(hidden_3)
output = Dense(answers_shape[1],activation='softmax')(dropout)
answers_network = Model(inputs=[questions,context],outputs=output)
answers_network.summary()

NameError: ignored

In [0]:
answers_network.compile("adam","binary_crossentropy",metrics=[f1])

#### Train the neural network.

In [0]:
answers_network_checkpoint = ModelCheckpoint('answers_network-non-rnn-best.h5', verbose=1, monitor='val_f1',save_best_only=True, mode='auto') 

In [0]:
print(answers_network.metrics_names)

In [0]:

answers_network.fit(x=[questionsTrain,contextTrain],y=Y_2_train_num,callbacks=[answers_network_checkpoint],validation_split=0.2,verbose=True,epochs=9)
#print("Weights: ",questions_article_model.get_weights())

#### Loading the model with best fit.

In [0]:
answers_network.load_weights('answers_network-non-rnn-best.h5')

In [0]:

answers_network.evaluate([questionsDev,contextDev],Y_2_dev_num)

## Transfer learning using Google's Bert

Our model has a very poor F1 score. Let's see if we can't build a better model. We'll use Google's BERT deep learning network, which is so good that the networks with SQuaD 2.0's highest recorded F1 score use it.  

The basic idea behind BERT is this: Neural networks rely on numerical vectors. Similar sentences and phrases should produce similar vectors.

"It is not possible to train bidirectional models by simply conditioning each word on words before and after it. Doing this would allow the word that’s being predicted to indirectly see itself in a multi-layer model. To solve this, Google researchers used a straightforward technique of masking out some words in the input and condition each word bidirectionally in order to predict the masked words. This idea is not new, but BERT is the first technique where it was successfully used to pre-train a deep neural network." (packtpub.com)

In [0]:
!pip install wget # Used for downloading the BERT model.
import wget
import os
import urllib
import tensorflow as tf
from tensorflow import keras
import zipfile

In [0]:
!git clone https://github.com/google-research/bert.git

fatal: destination path 'bert' already exists and is not an empty directory.


In [0]:
bert_model_name = "wwm_uncased_L-24_H-1024_A-16"

In [0]:
bert_model_file_name = bert_model_name + ".zip"
online_bert_path = "https://storage.googleapis.com/bert_models/2019_05_30/" + bert_model_file_name
if not os.path.isfile(bert_model_file_name):
  wget.download(online_bert_path)
  zipfile.ZipFile(bert_model_file_name,'r').extractall()
print(os.path.abspath(bert_model_name))

/content/wwm_uncased_L-24_H-1024_A-16



### The neural network.

In [0]:
os.chdir(currentDirectory)


In [0]:
# @title Preparation
!pip install -q keras-bert
!wget -q https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip -o uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [0]:
# @title Environment
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
checkpoint_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

# TF_KERAS must be added to environment variables in order to use TPU
os.environ['TF_KERAS'] = '1'

In [0]:
# @title Load Basic Model
import codecs
from keras_bert import load_trained_model_from_checkpoint
assert os.path.exists(pretrained_path), str(pretrained_path + " does not exist")
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)

model = load_trained_model_from_checkpoint(config_path, checkpoint_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, 512)          0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, 512)          0                                            
__________________________________________________________________________________________________
Embedding-Token (TokenEmbedding [(None, 512, 768), ( 23440896    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, 512, 768)     1536        Input-Segment[0][0]              
__________________________________________________________________________________________________
Embedding-

In [0]:
# @title Initialize Variables
import tensorflow as tf
import tensorflow.keras.backend as K

sess = K.get_session()
uninitialized_variables = set([i.decode('ascii') for i in sess.run(tf.report_uninitialized_variables())])
init_op = tf.variables_initializer(
    [v for v in tf.global_variables() if v.name.split(':')[0] in uninitialized_variables]
)
sess.run(init_op)

In [0]:
# @title Convert to TPU Model
import tensorflow as tf
from keras_bert import get_custom_objects

tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
strategy = tf.contrib.tpu.TPUDistributionStrategy(
    tf.contrib.cluster_resolver.TPUClusterResolver(tpu=tpu_address)
)

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    tpu_model = tf.contrib.tpu.keras_to_tpu_model(model, strategy=strategy)
    tpu_model.compile('adam', 'sparse_categorical_crossentropy')


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Querying Tensorflow master (grpc://10.0.84.154:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12467959691246213120)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 2858707863396374272)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 4037339612110226200)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:1,

In [0]:
# @title Extraction
import numpy as np
from keras_bert import Tokenizer

tokenizer = Tokenizer(token_dict)
text = 'From that day forth... my arm changed... and a voice echoed'
tokens = tokenizer.tokenize(text)
indices, segments = tokenizer.encode(first=text, max_len=512)
print(tokens)

with tf.keras.utils.custom_object_scope(get_custom_objects()):
    predicts = tpu_model.predict([np.array([indices] * 8), np.array([segments] * 8)])[0]
    
for i, token in enumerate(tokens):
    print(token, predicts[i].tolist()[:19])

['[CLS]', 'from', 'that', 'day', 'forth', '.', '.', '.', 'my', 'arm', 'changed', '.', '.', '.', 'and', 'a', 'voice', 'echoed', '[SEP]']
INFO:tensorflow:New input shapes; (re-)compiling: mode=infer (# of cores 8), [TensorSpec(shape=(1, 512), dtype=tf.float32, name='Input-Token_10'), TensorSpec(shape=(1, 512), dtype=tf.float32, name='Input-Segment_10')]
INFO:tensorflow:Overriding default placeholder.
INFO:tensorflow:Cloning Adam {'lr': 0.0010000000474974513, 'beta_1': 0.8999999761581421, 'beta_2': 0.9990000128746033, 'decay': 0.0, 'epsilon': 1e-07, 'amsgrad': False}
INFO:tensorflow:Remapping placeholder for Input-Token
INFO:tensorflow:Remapping placeholder for Input-Segment


TypeError: ignored

In [0]:

answers_network.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
lambda_8 (Lambda)               (None, 128)          0           input_10[0][0]                   
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, 128)          0           input_11[0][0]                   
__________________________________________________________________________________________________
dense_11 (

In [0]:
answers_network.compile("rmsprop","categorical_crossentropy",metrics=[f1])
answers_network_checkpoint = ModelCheckpoint('answers_network-rnn-best.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='auto')
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  y=session.run(tf.one_hot(Y_2_train,max_answer_start+1))
  print(X_2_train_text.shape)
  print(y.shape)
  x = np.hsplit(X_2_train_text,2)
  answers_network.fit(x=x,y=y,callbacks=[answers_network_checkpoint],epochs=8,validation_split=0.2)

(87599, 2)
(87599, 240)
Train on 70079 samples, validate on 17520 samples
Epoch 1/8
70016/70079 [============================>.] - ETA: 0s - loss: 2.6111 - f1: 4.6314e-04
Epoch 00001: val_loss improved from inf to 2.33924, saving model to answers_network-rnn-best.h5
70079/70079 [==============================] - 39s 551us/sample - loss: 2.6113 - f1: 4.6272e-04 - val_loss: 2.3392 - val_f1: 0.0000e+00
Epoch 2/8
70048/70079 [============================>.] - ETA: 0s - loss: 2.5859 - f1: 0.0013
Epoch 00002: val_loss did not improve from 2.33924
70079/70079 [==============================] - 38s 535us/sample - loss: 2.5856 - f1: 0.0013 - val_loss: 2.3703 - val_f1: 0.0000e+00
Epoch 3/8
69952/70079 [============================>.] - ETA: 0s - loss: 2.5735 - f1: 8.0789e-04
Epoch 00003: val_loss did not improve from 2.33924
70079/70079 [==============================] - 39s 556us/sample - loss: 2.5742 - f1: 8.0641e-04 - val_loss: 2.3829 - val_f1: 4.9543e-04
Epoch 4/8
69984/70079 [==============